<a href="https://colab.research.google.com/github/abubakarzohaib141/Chatbot1415/blob/main/Bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install google-generativeai
!pip install requests

In [22]:
import os
import google.generativeai as genai
import requests  # To call Tavily API

# 🔑 API Keys
gemini_api_key = "your_gemini_api_key_here"  # Replace this with your Gemini API Key
tavily_api_key = "tavily_api_Key_here"  # Replace this with your Tavily API Key

# Configure the genai library for Gemini
genai.configure(api_key=gemini_api_key)

# 🔥 **Short-Term Memory** to store user session data
conversation_memory = {}

def update_memory(user_id, key, value):
    """
    Updates the user's memory with a specific key-value pair.
    """
    if user_id not in conversation_memory:
        conversation_memory[user_id] = {}
    conversation_memory[user_id][key] = value

def get_user_memory(user_id, key):
    """
    Retrieves the memory for a specific user and key.
    """
    return conversation_memory.get(user_id, {}).get(key, None)

# 🔥 **Tavily API Integration**
def fetch_from_tavily(query):
    """
    Call the Tavily API to get relevant information.
    """
    try:
        url = f"https://api.tavily.com/search?query={query}&apikey={tavily_api_key}"
        response = requests.get(url)

        if response.status_code == 200:
            data = response.json()
            print(f"✅ Tavily Response: {data}")  # Debugging info
            return data.get('results', [])
        else:
            print(f"❌ Tavily API Error: {response.status_code} - {response.text}")
            return f"Sorry, I couldn't fetch information from Tavily for '{query}'."
    except Exception as e:
        print(f"❌ Error calling Tavily API: {e}")
        return "An error occurred while calling the Tavily API."

# 🔥 **Gemini Chat Session Setup**
generation_config = {
    "temperature": 1.55,
    "top_p": 0.95,
    "top_k": 40,
    "max_output_tokens": 8192,
    "response_mime_type": "text/plain",
}

# 🔥 **Start the Gemini Chat Session**
model = genai.GenerativeModel(
    model_name="gemini-1.5-flash",
    generation_config=generation_config,
    system_instruction=(
        "You are a helpful assistant. Your main task is to help users by answering questions, providing simple and clear explanations. "
        "If someone says 'search' or 'find', you should ask Tavily to search for the requested information. If the user asks you to 'summarize it', you should summarize the last message. "
        "Be polite, friendly, and accurate in your responses."
    ),
)

# **Start a chat session** (This will allow us to use send_message() correctly)
chat_session = model.start_chat(
    history=[]
)

# 🔥 **Chatbot Response Logic**
def chatbot_response(user_input, user_id="user_1"):
    """
    Handles user input, interacts with Gemini and Tavily, and returns a response.
    """
    try:
        # 🔥 Handle "search" or "find" using Tavily
        if "search" in user_input.lower() or "find" in user_input.lower():
            print(f"🔍 Searching Tavily for: {user_input}")  # Debug
            tavily_response = fetch_from_tavily(user_input)
            update_memory(user_id, 'last_tavily_response', tavily_response)
            if isinstance(tavily_response, list) and len(tavily_response) > 0:
                message_to_gemini = "Summarize the following cricket news articles:\n"
                for article in tavily_response[:5]:
                    message_to_gemini += f"Title: {article.get('title', 'No Title')}, Link: {article.get('url', 'No URL')}\n"
                response = chat_session.send_message(message_to_gemini)
                return f"🤖 **Summary from Gemini**: {response.text}"
            else:
                return "❌ No search results found from Tavily."

        # 🔥 Handle general queries using Gemini
        print(f"💬 Sending message to Gemini: {user_input}")  # Debug
        response = chat_session.send_message(user_input)
        print(f"🤖 Gemini Response: {response.text}")  # Debug

        # Store the user's input and Gemini's response in memory
        update_memory(user_id, 'last_user_input', user_input)
        update_memory(user_id, 'last_gemini_response', response.text)

        return response.text
    except Exception as e:
        print(f"❌ Error in chatbot_response: {e}")
        return "An error occurred while generating a response. Please try again later."

# 🔥 **Main Chat Loop**
try:
    print("Type 'q', 'exit', or 'quit' to end the chat.\n")
    while True:
        user_input = input("👤 You: ")

        # 🔥 Check if user wants to exit the chat
        if user_input.lower() in ["q", "exit", "quit"]:
            print("Exiting chat. Goodbye! 👋")
            break

        response = chatbot_response(user_input)
        print(f"🤖 ChatBot: {response}\n")
except Exception as e:
    print(f"❌ An error occurred: {e}")


Type 'q', 'exit', or 'quit' to end the chat.

👤 You: Hello
💬 Sending message to Gemini: Hello
🤖 Gemini Response: Hello there! How can I help you today?

🤖 ChatBot: Hello there! How can I help you today?


👤 You: exit
Exiting chat. Goodbye! 👋
